In [1]:

import numpy as np
import tensorflow as tf
import pickle

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from flask import Flask, request, jsonify
from sklearn.preprocessing import LabelEncoder

In [3]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# Load the model, tokenizer (words.pkl), and classes (classes.pkl)
model = tf.keras.models.load_model('D:\chatbot\chatbot_model.h5')

# Load tokenizer and label encoder
with open('D:\\chatbot\\words.pkl', 'rb') as f:
    words = pickle.load(f)

with open('D:\\chatbot\\classes.pkl', 'rb') as f:
    classes = pickle.load(f)


In [5]:
 #Initialize Flask app
app = Flask(__name__)

# Preprocess the user input
def preprocess_input(user_input):
    # Tokenize and encode the user input
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(words)
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = pad_sequences(sequence, maxlen=20, padding='post')  # Adjust padding if needed
    return padded_sequence

# Get the chatbot response
def chatbot_response(user_input):
    processed_input = preprocess_input(user_input)
    prediction = model.predict(processed_input)
    predicted_class = np.argmax(prediction)
    
    # Map prediction back to a class
    response_class = classes[predicted_class]  
    return response_class

# Flask route for chatbot interaction
@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get("message")
    if not user_input:
        return jsonify({"error": "No input provided"}), 400
    
    response = chatbot_response(user_input)
    return jsonify({"response": response})
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, this is the homepage of your chatbot!"

# Your existing chatbot code here...

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
  # Bind to all network interfaces

   


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Nov/2024 09:39:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2024 09:39:38] "GET /favicon.ico HTTP/1.1" 404 -


In [7]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import pickle
import tensorflow as tf
import json
import re
from sympy import symbols, diff, integrate, log, exp, sympify
import sympy as sp

# Define symbols for calculus operations
x = symbols('x')

# Function to handle basic arithmetic calculations
def calculate_math(expression):
    try:
        match = re.match(r"(\d+)\s*([+\-*/])\s*(\d+)", expression)
        if match:
            num1 = int(match.group(1))
            operator = match.group(2)
            num2 = int(match.group(3))

            if operator == '+':
                return num1 + num2
            elif operator == '-':
                return num1 - num2
            elif operator == '*':
                return num1 * num2
            elif operator == '/':
                return num1 / num2
        else:
            return "Sorry, I couldn't understand the calculation."
    except Exception as e:
        return "There was an error with the calculation."

# Function for advanced math operations (differentiation, integration, log, exponentials)
def calculate_expression(user_message):
    try:
        # Differentiation
        diff_match = re.search(r'differentiate\s(.+)', user_message, re.IGNORECASE)
        if diff_match:
            expression = sympify(diff_match.group(1))
            result = diff(expression, x)
            return f"The derivative of {expression} with respect to x is {result}"

        # Integration
        int_match = re.search(r'integrate\s(.+)', user_message, re.IGNORECASE)
        if int_match:
            expression = sympify(int_match.group(1))
            result = integrate(expression, x)
            return f"The integral of {expression} with respect to x is {result}"

        # Logarithm
        log_match = re.search(r'log\s*\(\s*(\d+)\s*\)', user_message, re.IGNORECASE)
        if log_match:
            number = int(log_match.group(1))
            result = log(number)
            return f"The logarithm of {number} is {result}"

        # Exponential
        exp_match = re.search(r'e\^\s*(\d+)', user_message, re.IGNORECASE)
        if exp_match:
            power = int(exp_match.group(1))
            result = sp.exp(power).evalf()  # Evaluate the exponential as a float
            return f"The result of e^{power} is {result}"

        return "Sorry, I couldn't understand the calculation."
    except Exception as e:
        return "There was an error with the calculation."

# Load the chatbot model and necessary files
model = tf.keras.models.load_model(r'D:\chatbot\chatbot_model.h5')
words = pickle.load(open(r'D:\chatbot\words.pkl', 'rb'))
classes = pickle.load(open(r'D:\chatbot\classes.pkl', 'rb'))

# Load intents.json with utf-8 encoding
with open(r'D:\chatbot\intents.json', encoding='utf-8') as file:
    intents = json.load(file)

app = Flask(__name__)

# Homepage route
@app.route('/')
def home():
    return render_template('home.html')

# Chatbot page route
@app.route('/chatbot')
def chatbot():
    return render_template('chat.html')

# Route to get chatbot responses
@app.route('/get_response', methods=['POST'])
def get_response():
    user_message = request.json['message']
    
    # Check if the user message is an advanced math question
    advanced_math_response = calculate_expression(user_message)
    if "Sorry" not in advanced_math_response:
        return jsonify({"response": advanced_math_response})
    
    # Check if the user message is a basic math question
    if re.search(r'\d+\s*[+\-*/]\s*\d+', user_message):
        result = calculate_math(user_message)
        return jsonify({"response": f"The answer is {result}"})
    
    # Process regular chatbot responses
    response = generate_chatbot_response(user_message)
    return jsonify({'response': response})

# Function to process user input and return chatbot response
def generate_chatbot_response(message):
    message_tokens = tokenize_sentence(message)
    prediction = model.predict(np.array([message_tokens]))[0]
    predicted_class = np.argmax(prediction)
    response = get_response_for_class(predicted_class)
    return response

# Tokenize the sentence and convert it into a bag of words
def tokenize_sentence(sentence):
    sentence_words = sentence.lower().split()
    bag_of_words = [0] * len(words)
    for word in sentence_words:
        if word in words:
            index = words.index(word)
            bag_of_words[index] = 1
    return np.array(bag_of_words)

# Function to get the response based on predicted class
def get_response_for_class(predicted_class):
    intent_label = classes[predicted_class]
    for intent in intents['intents']:
        if intent['tag'] == intent_label:
            return np.random.choice(intent['responses'])
    return "Sorry, I don't understand that."

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [18]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import pickle
import tensorflow as tf
import json
import re
from sympy import symbols, diff, integrate, log, exp
import sympy as sp

# Define symbols for calculus operations
x = symbols('x')

# Function to handle basic arithmetic calculations
def calculate_math(expression):
    try:
        match = re.match(r"(\d+)\s*([+\-*/])\s*(\d+)", expression)
        if match:
            num1 = int(match.group(1))
            operator = match.group(2)
            num2 = int(match.group(3))

            if operator == '+':
                return num1 + num2
            elif operator == '-':
                return num1 - num2
            elif operator == '*':
                return num1 * num2
            elif operator == '/':
                return num1 / num2
        else:
            return "Sorry, I couldn't understand the calculation."
    except Exception as e:
        return "There was an error with the calculation."

# Function for advanced math operations (differentiation, integration, log, exponentials)
def calculate_expression(user_message):
    try:
        # Differentiation
        diff_match = re.search(r'differentiate\s(.+)', user_message, re.IGNORECASE)
        if diff_match:
            expression = sp.sympify(diff_match.group(1))
            result = diff(expression, x)
            return f"The derivative of {expression} with respect to x is {result}"

        # Integration
        int_match = re.search(r'integrate\s(.+)', user_message, re.IGNORECASE)
        if int_match:
            expression = sp.sympify(int_match.group(1))
            result = integrate(expression, x)
            return f"The integral of {expression} with respect to x is {result}"

        # Logarithm
        log_match = re.search(r'log\s*\(\s*(\d+)\s*\)', user_message, re.IGNORECASE)
        if log_match:
            number = int(log_match.group(1))
            result = log(number)
            return f"The logarithm of {number} is {result}"

        # Exponential
        exp_match = re.search(r'e\^\s*(\d+)', user_message, re.IGNORECASE)
        if exp_match:
            power = int(exp_match.group(1))
            result = exp(power)
            return f"The result of e^{power} is {result}"

        return "Sorry, I couldn't understand the calculation."
    except Exception as e:
        return "There was an error with the calculation."

# Load the chatbot model and necessary files
model = tf.keras.models.load_model(r'D:\chatbot\chatbot_model.h5')
words = pickle.load(open(r'D:\chatbot\words.pkl', 'rb'))
classes = pickle.load(open(r'D:\chatbot\classes.pkl', 'rb'))

# Load intents.json with utf-8 encoding
with open(r'D:\chatbot\intents.json', encoding='utf-8') as file:
    intents = json.load(file)

app = Flask(__name__)

# Homepage route
@app.route('/')
def home():
    return render_template('home.html')

# Chatbot page route
@app.route('/chatbot')
def chatbot():
    return render_template('chat.html')

# Route to get chatbot responses
@app.route('/get_response', methods=['POST'])
def get_response():
    user_message = request.json['message']
    
    # Check if the user message is an advanced math question
    advanced_math_response = calculate_expression(user_message)
    if "Sorry" not in advanced_math_response:
        return jsonify({"response": advanced_math_response})
    
    # Check if the user message is a basic math question
    if re.search(r'\d+\s*[+\-*/]\s*\d+', user_message):
        result = calculate_math(user_message)
        return jsonify({"response": f"The answer is {result}"})
    
    # Process regular chatbot responses
    response = generate_chatbot_response(user_message)
    return jsonify({'response': response})

# Function to process user input and return chatbot response
def generate_chatbot_response(message):
    message_tokens = tokenize_sentence(message)
    prediction = model.predict(np.array([message_tokens]))[0]
    predicted_class = np.argmax(prediction)
    response = get_response_for_class(predicted_class)
    return response

# Tokenize the sentence and convert it into a bag of words
def tokenize_sentence(sentence):
    sentence_words = sentence.lower().split()
    bag_of_words = [0] * len(words)
    for word in sentence_words:
        if word in words:
            index = words.index(word)
            bag_of_words[index] = 1
    return np.array(bag_of_words)

# Function to get the response based on predicted class
def get_response_for_class(predicted_class):
    intent_label = classes[predicted_class]
    for intent in intents['intents']:
        if intent['tag'] == intent_label:
            return np.random.choice(intent['responses'])
    return "Sorry, I don't understand that."

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Nov/2024 10:36:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2024 10:36:45] "GET /chatbot HTTP/1.1" 200 -


1/1 [==============================] - 0s 161ms/step


127.0.0.1 - - [13/Nov/2024 10:36:55] "POST /get_response HTTP/1.1" 200 -
